In [90]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from splinter import Browser
from pprint import pprint
import pymongo
import time

In [91]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [92]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [93]:
# Define database and collection
db = client.mars_db
collection = db.mars_data

## Scrape Mars News from NASA

In [100]:
# Create BeautifulSoup object; parse with 'html.parser'
# Obtain the title and append Title list

news_url = "https://mars.nasa.gov/news/?page=0"
browser.visit(news_url)
image = browser.html

# HTML Object
html = browser.html

# Soup
news_soup = BeautifulSoup(html, 'html.parser')


# Retrieve the latest element that contains news 
news_title = news_soup.find('div', class_='content_title').find('a').text
news_p = news_soup.find('div', class_='article_teaser_body').text

#Print Data to confirm
print(news_title)
print(news_p)



After a Reset, Curiosity Is Operating Normally
NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. 


## Scrape JPL Featured Image

In [95]:
#Scrape JPL Images (FUCK YOU I LOST MY FIRST DRAFT)

jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
image = browser.html

image_soup = BeautifulSoup(image, 'html.parser')
featured_image = image_soup.find('article')['style']
featured_image_url = jpl_url + featured_image.replace("background-image: url('",'').replace("');",'')

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA11777-1920x1200.jpg'

## Scrape Weather

In [96]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

weather = browser.html
weather_soup = BeautifulSoup(weather, "html.parser")

mars_weather = weather_soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)


Curiosity is again operating normally following a boot problem first experienced last Friday. Look for more Gale Crater weather conditions soon.
https://www.jpl.nasa.gov/news/news.php?feature=7339 …pic.twitter.com/gFMfXyeWDa


## Scrape Facts

In [ ]:
#Variable for Mars facts
#facts = pd.read_html(str("http://space-facts.com/mars/"))
#facts

## Scrape hemisphere data

In [22]:
astrogeo_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser.visit(astrogeo_url)
html_astro = browser.html

astrogeo_content = BeautifulSoup(html_astro, "html.parser")
astrogeo_content
items = astrogeo_content.find_all('div', class_='item')

hemisphere_image_urls = []

# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
hemispheres = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_url = 'https://astrogeology.usgs.gov'

# Loop through the hemispheres
for hemisphere in hemispheres: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls
    

[{'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

## Create dictionary for Mongo_db

In [97]:
 mars_entry = {
    'title': news_title,
    'text': news_text,
    'image_url': featured_image_url,
    'weather': mars_weather,
    'hemispher_images': hemisphere_image_urls
        }

In [98]:
mars_entry

{'title': 'After a Reset, Curiosity Is Operating Normally',
 'text': {},
 'image_url': 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA11777-1920x1200.jpg',
 'weather': 'Curiosity is again operating normally following a boot problem first experienced last Friday. Look for more Gale Crater weather conditions soon.\nhttps://www.jpl.nasa.gov/news/news.php?feature=7339\xa0…pic.twitter.com/gFMfXyeWDa',
 'hemispher_images': [{'title': 'Valles Marineris Hemisphere Enhanced',
   'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
  {'title': 'Valles Marineris Hemisphere Enhanced',
   'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
  {'title': 'Valles Marineris Hemisphere Enhanced',
   'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enha

In [99]:
pprint(mars_entry)

{'hemispher_images': [{'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
                       'title': 'Valles Marineris Hemisphere Enhanced'},
                      {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
                       'title': 'Valles Marineris Hemisphere Enhanced'},
                      {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
                       'title': 'Valles Marineris Hemisphere Enhanced'},
                      {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
                       'title': 'Valles Marineris Hemisphere Enhanced'}],
 'image_url': 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallp